In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
# Import the API key
geoapify_key = "ad238f1c0dbc44b69a669cc677dbd624"

In [2]:
# Single City Example For Reference

# API_KEY = "ad238f1c0dbc44b69a669cc677dbd624"
# city = "Toronto, Ontario"

# # Step 1: Geocode city
# geocode_url = f"https://api.geoapify.com/v1/geocode/search?text={city}&apiKey={API_KEY}"
# geocode_response = requests.get(geocode_url)
# geocode_data = geocode_response.json()

# # Get coordinates
# location = geocode_data['features'][0]['geometry']['coordinates']
# lon, lat = location

# # Step 2: Search for hospitals near the city center
# radius = 10000  # in meters (10 km)
# toronto_hospitals_url = f"https://api.geoapify.com/v2/places?categories=healthcare.hospital&filter=circle:{lon},{lat},{radius}&limit=20&apiKey={API_KEY}"

# toronto_hospitals_response = requests.get(toronto_hospitals_url)
# toronto_hospitals_data = toronto_hospitals_response.json()

# # Print hospital names
# print("Hospitals in", city)
# for feature in toronto_hospitals_data['features']:
#     name = feature['properties'].get('name', 'Unnamed hospital')
#     print("-", name)


In [3]:
API_KEY = "ad238f1c0dbc44b69a669cc677dbd624"
cities = ["Toronto", "Montreal", "Calgary", "Ottawa", "Edmonton", "Winnipeg", "Mississauga", "Vancouver", "Brampton", "Hamilton"]
cities_no_space = []

for city in cities:
    cities_no_space.append(city.replace(" ", "%20"))
    cities_url = []
    cities_response = []
    cities_geocode_data = []

# step 1: geocode city
for city in cities_no_space:
    geocode_url = f"https://api.geoapify.com/v1/geocode/search?text={city}&apiKey={API_KEY}"
    cities_url.append(geocode_url)
    geocode_response = requests.get(geocode_url)
    cities_response.append(geocode_response)
    geocode_data = geocode_response.json()
    cities_geocode_data.append(geocode_data)

pprint(cities_geocode_data)

[{'features': [{'bbox': [-79.492821, 43.610384, -79.2785104, 43.7360196],
                'geometry': {'coordinates': [-79.3839347, 43.6534817],
                             'type': 'Point'},
                'properties': {'address_line1': 'Old Toronto, ON',
                               'address_line2': 'Canada',
                               'category': 'administrative',
                               'city': 'Old Toronto',
                               'country': 'Canada',
                               'country_code': 'ca',
                               'county': 'Golden Horseshoe',
                               'datasource': {'attribution': '© OpenStreetMap '
                                                             'contributors',
                                              'license': 'Open Database '
                                                         'License',
                                              'sourcename': 'openstreetmap',
                          

In [4]:
# Get Coordinates
city_data = {}

for x in range(0,len(cities)):
    city_data[cities[x]] = {"location" : cities_geocode_data[x]['features'][0]['geometry']['coordinates']}

pprint(city_data)

{'Brampton': {'location': [-79.7599366, 43.685832]},
 'Calgary': {'location': [-114.057541, 51.0456064]},
 'Edmonton': {'location': [-113.491241, 53.5462055]},
 'Hamilton': {'location': [-79.8728583, 43.2560802]},
 'Mississauga': {'location': [-79.6443879, 43.5896231]},
 'Montreal': {'location': [-73.5698065, 45.5031824]},
 'Ottawa': {'location': [-75.6901106, 45.4208777]},
 'Toronto': {'location': [-79.3839347, 43.6534817]},
 'Vancouver': {'location': [-123.113952, 49.2608724]},
 'Winnipeg': {'location': [-97.1384584, 49.8955367]}}


In [5]:
# Step 2: Search for hospitals near the city center
radius = 10000 #in meters (10km)

for city in cities:
    lon, lat = city_data[city]["location"]
    hospitals_arr = []
    hospitals_locations = []
    hospitals_url = f"https://api.geoapify.com/v2/places?categories=healthcare.hospital&filter=circle:{lon},{lat},{radius}&limit=20&apiKey={API_KEY}"
    hospitals_response = requests.get(hospitals_url)
    hospitals_data = hospitals_response.json()
    for feature in hospitals_data['features']:
        name = feature['properties'].get('name', 'Unnamed hospital')
        longitude = feature["properties"].get("lon", "NA")
        latitude = feature["properties"].get("lat", "NA")
        hospitals_arr.append(name)
        hospitals_locations.append([longitude, latitude])
        city_data[city]["hospitals_count"] = len(hospitals_arr)
        city_data[city]["hospitals"] = hospitals_arr
        city_data[city]["hospitals_locations"] = hospitals_locations

pprint(city_data)

{'Brampton': {'hospitals': ['Brampton Civic Hospital',
                            'Peel Memorial Centre for Integrated Health and '
                            'Wellness'],
              'hospitals_count': 2,
              'hospitals_locations': [[-79.74383437758848, 43.7469379],
                                      [-79.75124173364408, 43.69088135]],
              'location': [-79.7599366, 43.685832]},
 'Calgary': {'hospitals': ['Foothills Medical Centre',
                           'Peter Lougheed Centre of the Calgary General '
                           'Hospital',
                           'Sheldon M. Chumir Health Centre',
                           "Alberta Children's Hospital",
                           'Rockyview General Hospital',
                           'Rockyview General Hospital EMERGENCY ENTRANCE',
                           'Foothills Medical Centre EMERGENCY ENTRANCE',
                           'Harvest Medicine Calgary & Alberta',
                           'Br

In [6]:
cities_hospitals_df = pd.DataFrame(city_data)
cities_hospitals_df = cities_hospitals_df.transpose()
cities_hospitals_df.head()

,location,hospitals_count,hospitals,hospitals_locations
Toronto,"[-79.3839347, 43.6534817]",20,"[St. Michael's Hospital, Sunnybrook Health Sci...","[[-79.37780684414469, 43.6536408], [-79.375541..."
Montreal,"[-73.5698065, 45.5031824]",20,[Institut universitaire en santé mentale de Mo...,"[[-73.53062953615134, 45.58879855], [-73.56349..."
Calgary,"[-114.057541, 51.0456064]",9,"[Foothills Medical Centre, Peter Lougheed Cent...","[[-114.13505565301116, 51.06541815], [-113.984..."
Ottawa,"[-75.6901106, 45.4208777]",12,"[Children's Hospital of Eastern Ontario, Hôpit...","[[-75.65146146979367, 45.40148025], [-75.63940..."
Edmonton,"[-113.491241, 53.5462055]",20,"[Glenrose Rehabilitation Hospital, Royal Alexa...","[[-113.49834793893291, 53.56024635], [-113.497..."


In [7]:
# END OF GETTING HOSPITAL DATA

In [8]:
# GET SCHOOL DATA

In [9]:
# Single City Example For Reference

# # Single City Example For Reference

# API_KEY = "ad238f1c0dbc44b69a669cc677dbd624"
# city = "Toronto, Ontario"

# # Step 1: Geocode city
# geocode_url = f"https://api.geoapify.com/v1/geocode/search?text={city}&apiKey={API_KEY}"
# geocode_response = requests.get(geocode_url)
# geocode_data = geocode_response.json()

# # Get coordinates
# location = geocode_data['features'][0]['geometry']['coordinates']
# lon, lat = location

# # Step 2: Search for universities near the city center
# radius = 10000  # in meters (10 km)
# toronto_universities_url = f"https://api.geoapify.com/v2/places?categories=education.university&filter=circle:{lon},{lat},{radius}&limit=20&apiKey={API_KEY}"
# toronto_universities_response = requests.get(toronto_universities_url)
# toronto_universities_data = toronto_universities_response.json()

# # Print university names
# print("Universities in", city)
# for feature in toronto_universities_data['features']:
#     name = feature['properties'].get('name', 'Unnamed university')
#     print("-", name)


In [10]:
# Search for universities near the city center
radius = 10000 #in meters (10km)

for city in cities:
    lon, lat = city_data[city]["location"]
    universities_arr = []
    universities_locations = []
    universities_url = f"https://api.geoapify.com/v2/places?categories=education.university&filter=circle:{lon},{lat},{radius}&limit=20&apiKey={API_KEY}"
    universities_response = requests.get(universities_url)
    universities_data = universities_response.json()
    for feature in universities_data['features']:
        name = feature['properties'].get('name', 'Unnamed hospital')
        longitude = feature["properties"].get("lon", "NA")
        latitude = feature["properties"].get("lat", "NA")
        universities_arr.append(name)
        universities_locations.append([longitude, latitude])
        city_data[city]["universities_count"] = len(universities_arr)
        city_data[city]["universities"] = universities_arr
        city_data[city]["universities_locations"] = universities_locations

pprint(city_data)

{'Brampton': {'hospitals': ['Brampton Civic Hospital',
                            'Peel Memorial Centre for Integrated Health and '
                            'Wellness'],
              'hospitals_count': 2,
              'hospitals_locations': [[-79.74383437758848, 43.7469379],
                                      [-79.75124173364408, 43.69088135]],
              'location': [-79.7599366, 43.685832],
              'universities': ['Algoma at Brampton', 'Algoma University'],
              'universities_count': 2,
              'universities_locations': [[-79.7594415, 43.6866755],
                                         [-79.7603715, 43.6864357]]},
 'Calgary': {'hospitals': ['Foothills Medical Centre',
                           'Peter Lougheed Centre of the Calgary General '
                           'Hospital',
                           'Sheldon M. Chumir Health Centre',
                           "Alberta Children's Hospital",
                           'Rockyview General Hospi

In [11]:
cities_df = pd.DataFrame(city_data)
cities_df = cities_df.transpose()
cities_df

,location,hospitals_count,hospitals,hospitals_locations,universities_count,universities,universities_locations
Toronto,"[-79.3839347, 43.6534817]",20,"[St. Michael's Hospital, Sunnybrook Health Sci...","[[-79.37780684414469, 43.6536408], [-79.375541...",13,"[University of Toronto, Toronto Metropolitan U...","[[-79.39775966248058, 43.663461999999996], [-7..."
Montreal,"[-73.5698065, 45.5031824]",20,[Institut universitaire en santé mentale de Mo...,"[[-73.53062953615134, 45.58879855], [-73.56349...",20,"[Université de Montréal, HEC Montréal, McGill ...","[[-73.61454175576318, 45.50475765], [-73.62212..."
Calgary,"[-114.057541, 51.0456064]",9,"[Foothills Medical Centre, Peter Lougheed Cent...","[[-114.13505565301116, 51.06541815], [-113.984...",6,"[University of Calgary, Mount Royal University...","[[-114.13876168856827, 51.07505395], [-114.130..."
Ottawa,"[-75.6901106, 45.4208777]",12,"[Children's Hospital of Eastern Ontario, Hôpit...","[[-75.65146146979367, 45.40148025], [-75.63940...",8,"[Carleton University, University of Ottawa, Do...","[[-75.69500411688679, 45.3858581], [-75.683390..."
Edmonton,"[-113.491241, 53.5462055]",20,"[Glenrose Rehabilitation Hospital, Royal Alexa...","[[-113.49834793893291, 53.56024635], [-113.497...",6,"[University of Alberta, Campus St-Jean, Buildi...","[[-113.5244937350756, 53.52682], [-113.4698526..."
Winnipeg,"[-97.1384584, 49.8955367]",14,"[Concordia Hospital, Saint Boniface General Ho...","[[-97.06478436603102, 49.91344049999999], [-97...",9,"[Université de Saint-Boniface, University of W...","[[-97.11967426330207, 49.88911935], [-97.15388..."
Mississauga,"[-79.6443879, 43.5896231]",5,"[Credit Valley Hospital, Mississauga Hospital,...","[[-79.70371211203724, 43.559441], [-79.6080843...",1,[University of Toronto Mississauga],"[[-79.66261979322708, 43.54796195]]"
Vancouver,"[-123.113952, 49.2608724]",11,"[Burnaby Hospital, St. Paul's Hospital, Vancou...","[[-123.0154532657107, 49.24965685], [-123.1289...",16,"[UBC Learning Exchange, UBC Robson Square, Uni...","[[-123.09929373609779, 49.27920095], [-123.121..."
Brampton,"[-79.7599366, 43.685832]",2,"[Brampton Civic Hospital, Peel Memorial Centre...","[[-79.74383437758848, 43.7469379], [-79.751241...",2,"[Algoma at Brampton, Algoma University]","[[-79.7594415, 43.6866755], [-79.7603715, 43.6..."
Hamilton,"[-79.8728583, 43.2560802]",9,"[Joseph Brant Hospital, Juravinski Cancer Cent...","[[-79.80274912474499, 43.3171722], [-79.845632...",4,"[McMaster University, Redeemer University, McM...","[[-79.917825231868, 43.263939699999995], [-79...."


In [12]:
# END OF GETTING UNIVERSITY DATA

In [13]:
# GET SUPERMARKET DATA

In [14]:
# Search for supermarkets near the city center
radius = 10000 #in meters (10km)

for city in cities:
    lon, lat = city_data[city]["location"]
    supermarkets_arr = []
    supermarkets_locations = []
    supermarkets_url = f"https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=circle:{lon},{lat},{radius}&limit=20&apiKey={API_KEY}"
    supermarkets_response = requests.get(supermarkets_url)
    supermarkets_data = supermarkets_response.json()
    for feature in supermarkets_data['features']:
        name = feature['properties'].get('name', 'Unnamed supermarket')
        longitude = feature["properties"].get("lon", "NA")
        latitude = feature["properties"].get("lat", "NA")
        supermarkets_arr.append(name)
        supermarkets_locations.append([longitude, latitude])
        city_data[city]["supermarkets_count"] = len(supermarkets_arr)
        city_data[city]["supermarkets"] = supermarkets_arr
        city_data[city]["supermarkets_locations"] = supermarkets_locations

pprint(city_data)

{'Brampton': {'hospitals': ['Brampton Civic Hospital',
                            'Peel Memorial Centre for Integrated Health and '
                            'Wellness'],
              'hospitals_count': 2,
              'hospitals_locations': [[-79.74383437758848, 43.7469379],
                                      [-79.75124173364408, 43.69088135]],
              'location': [-79.7599366, 43.685832],
              'supermarkets': ['Walmart Supercentre',
                               'Walmart Supercentre',
                               'Walmart Supercentre',
                               'Ample Food Market',
                               'Indian Foodland',
                               '1 Stop African Caribbean Market',
                               "India Bazaar Fresh Pik's",
                               'No Frills',
                               'Walmart Supercentre',
                               'Sobeys',
                               'Walmart Supercentre',
          

In [15]:
cities_df = pd.DataFrame(city_data)
cities_df = cities_df.transpose()
cities_df

,location,hospitals_count,hospitals,hospitals_locations,universities_count,universities,universities_locations,supermarkets_count,supermarkets,supermarkets_locations
Toronto,"[-79.3839347, 43.6534817]",20,"[St. Michael's Hospital, Sunnybrook Health Sci...","[[-79.37780684414469, 43.6536408], [-79.375541...",13,"[University of Toronto, Toronto Metropolitan U...","[[-79.39775966248058, 43.663461999999996], [-7...",20,"[Great Food Supermarket, Real Canadian Superst...","[[-79.3008100311174, 43.71870199999999], [-79...."
Montreal,"[-73.5698065, 45.5031824]",20,[Institut universitaire en santé mentale de Mo...,"[[-73.53062953615134, 45.58879855], [-73.56349...",20,"[Université de Montréal, HEC Montréal, McGill ...","[[-73.61454175576318, 45.50475765], [-73.62212...",20,"[Metro, Kim Phat, Costco, Provigo, Maxi, Metro...","[[-73.46754094764486, 45.4565319], [-73.605678..."
Calgary,"[-114.057541, 51.0456064]",9,"[Foothills Medical Centre, Peter Lougheed Cent...","[[-114.13505565301116, 51.06541815], [-113.984...",6,"[University of Calgary, Mount Royal University...","[[-114.13876168856827, 51.07505395], [-114.130...",20,"[Calgary Co-op Dalhousie Centre, Real Canadian...","[[-114.14635434125222, 51.10284365], [-113.984..."
Ottawa,"[-75.6901106, 45.4208777]",12,"[Children's Hospital of Eastern Ontario, Hôpit...","[[-75.65146146979367, 45.40148025], [-75.63940...",8,"[Carleton University, University of Ottawa, Do...","[[-75.69500411688679, 45.3858581], [-75.683390...",20,"[McExcavate, Costco, FreshCo, Bulk Barn, Metro...","[[-75.6712828, 45.3817769], [-75.7222840229937..."
Edmonton,"[-113.491241, 53.5462055]",20,"[Glenrose Rehabilitation Hospital, Royal Alexa...","[[-113.49834793893291, 53.56024635], [-113.497...",6,"[University of Alberta, Campus St-Jean, Buildi...","[[-113.5244937350756, 53.52682], [-113.4698526...",20,"[Forum Law LLP, Safeway, Real Canadian Superst...","[[-113.5774657, 53.5711513], [-113.51357822124..."
Winnipeg,"[-97.1384584, 49.8955367]",14,"[Concordia Hospital, Saint Boniface General Ho...","[[-97.06478436603102, 49.91344049999999], [-97...",9,"[Université de Saint-Boniface, University of W...","[[-97.11967426330207, 49.88911935], [-97.15388...",20,"[Sobeys, Safeway, Costco, Safeway, Family Food...","[[-97.09479846745504, 49.838668], [-97.2014662..."
Mississauga,"[-79.6443879, 43.5896231]",5,"[Credit Valley Hospital, Mississauga Hospital,...","[[-79.70371211203724, 43.559441], [-79.6080843...",1,[University of Toronto Mississauga],"[[-79.66261979322708, 43.54796195]]",20,"[Real Canadian Superstore, Walmart Supercentre...","[[-79.63356041580671, 43.567600299999995], [-7..."
Vancouver,"[-123.113952, 49.2608724]",11,"[Burnaby Hospital, St. Paul's Hospital, Vancou...","[[-123.0154532657107, 49.24965685], [-123.1289...",16,"[UBC Learning Exchange, UBC Robson Square, Uni...","[[-123.09929373609779, 49.27920095], [-123.121...",20,"[Costco, Real Canadian Superstore, Costco, Wal...","[[-123.1217422673355, 49.19294575], [-123.0366..."
Brampton,"[-79.7599366, 43.685832]",2,"[Brampton Civic Hospital, Peel Memorial Centre...","[[-79.74383437758848, 43.7469379], [-79.751241...",2,"[Algoma at Brampton, Algoma University]","[[-79.7594415, 43.6866755], [-79.7603715, 43.6...",20,"[Walmart Supercentre, Walmart Supercentre, Wal...","[[-79.78371638858864, 43.705050650000004], [-7..."
Hamilton,"[-79.8728583, 43.2560802]",9,"[Joseph Brant Hospital, Juravinski Cancer Cent...","[[-79.80274912474499, 43.3171722], [-79.845632...",4,"[McMaster University, Redeemer University, McM...","[[-79.917825231868, 43.263939699999995], [-79....",20,"[No Frills, Denninger's Foods of the World, Fo...","[[-79.85236964039808, 43.251815050000005], [-7..."


In [16]:
# END OF GETTING SUPERMARKET DATA

In [ ]:
cities_df.to_csv("../Output/city_amenities_data.csv")
print("✅ Amenities and location data saved as 'city_amenities_data.csv'")

✅ Amenities and location data saved as 'city_amenities_data.csv'
